In [161]:
# Load data
import pandas as pd

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
data = pd.read_csv('q1.csv', parse_dates=['ts'], date_parser=dateparse)
data.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [9]:
# check if there are NaN in data
data.isnull().sum(axis=0)

ts            0
user_id       0
country_id    0
site_id       0
dtype: int64

In [10]:
data.describe()

,ts,user_id,country_id,site_id
count,3553,3553,3553,3553
unique,3538,1916,7,8
top,2019-02-03 20:00:34,LC3A59,TL6,5NPAU
freq,3,149,1449,1614
first,2019-02-01 00:01:24,NaN,NaN,NaN
last,2019-02-07 23:59:37,NaN,NaN,NaN


In [19]:
"""
Consider only the rows with country_id = "BDV" (there are 844 such rows). 
For each site_id, we can compute the number of unique user_id's found in these 844 rows. 
Which site_id has the largest number of unique users? And what's the number?
"""

bdv = data[data.country_id == 'BDV']
bdv.groupby(['site_id']).user_id.nunique()

site_id
3POLC      2
5NPAU    544
N0OTG     90
Name: user_id, dtype: int64

In [162]:
"""
Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, 
there are four users who visited a certain site more than 10 times.
Find these four users & which sites they (each) visited more than 10 times. 
Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.
"""

from datetime import datetime
import operator

timestamp1 = "2019-02-03 00:00:00"
timestamp2 = "2019-02-04 23:59:59"

t1 = datetime.strptime(timestamp1, "%Y-%m-%d %H:%M:%S")
t2 = datetime.strptime(timestamp2, "%Y-%m-%d %H:%M:%S")

selected_rows = data.loc[operator.and_(data.ts>=t1, data.ts<=t2)]
visit_count = selected_rows.groupby(["user_id", "site_id"]).country_id.count().reset_index()
visit_count_df = pd.DataFrame(visit_count)
visit_count_df.columns = ["user_id", "site_id", "visit_count"]
visit_count_df[visit_count_df.visit_count>10]

,user_id,site_id,visit_count
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


In [144]:
"""
For each site, compute the unique number of users whose last visit 
(found in the original data set) was to that site. 
For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. 
Based on this measure, what are top three sites? 
(hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; 
simply provide three pairs in the form (site_id, number of users).)
"""

last_visit = data.sort_values('ts').groupby('user_id').last()
last_visit_df = pd.DataFrame(last_visit.groupby('site_id').ts.count())
last_visit_df.sort_values('ts', ascending=False)

,ts
site_id,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


In [164]:
"""
For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. 
Compute the number of users whose first/last visits are to the same website. 
What is the number?
"""

first_visit = data.sort_values('ts').groupby('user_id').first()
users = first_visit[['site_id']]
users.columns = ['first_visit_site_id']

users = users.assign(last_visit_site_id = last_visit['site_id'])
users['same_first_last_site'] = users.apply(lambda x: x.first_visit_site_id == x.last_visit_site_id, axis=1)
print(users.same_first_last_site.sum())
users.tail()

1670


,first_visit_site_id,last_visit_site_id,same_first_last_site
user_id,,,
LCFC3B,N0OTG,N0OTG,True
LCFC3D,N0OTG,N0OTG,True
LCFC3E,5NPAU,5NPAU,True
LCFEC3,N0OTG,3POLC,False
LCFFC3,N0OTG,N0OTG,True
